In [5]:
import gspread

In [8]:
# connect to the service account
gc = gspread.service_account(filename="../creds.json")

sh = gc.open("Paper Extraction Evaluation").sheet1
sh.update_cell(10, 3, "helloooo")

FALSE
None


{'spreadsheetId': '1KunlZNe9InDjmYCOpgj4wgIAScObBxh237bCOpUcUCQ',
 'updatedRange': 'Sheet1!C10',
 'updatedRows': 1,
 'updatedColumns': 1,
 'updatedCells': 1}

In [1]:
import re

import sys

sys.path.append("../code")

from util import extract_doi_to_txt, get_string_from_text_file

In [2]:
pattern = "Results|Methodology|Methods"

In [3]:
txt = "The rain in Spain. The Spain"
x = re.search("^The.*Spain$", txt)
print(x)
y = re.findall("Spain", txt)
print(y)

<re.Match object; span=(0, 28), match='The rain in Spain. The Spain'>
['Spain', 'Spain']


In [96]:
# ***Last line should not contain "|"***
regex_method = r"""
    ^\d?[.]?\s?Method(s)?(ology)?$|# "method" "methods" "methodology"
    ^\d?[.]?\s?Material(s)?((\s)and(\s)Methods)?$ # "materials and methods" "material" "materials"
"""

# ***Last line should not contain "|"***
regex_result_only = r"""
    ^\d?[.]?\s?Result(s)?((\s)and(\s)discussion)?$ # "result" "results" "results and discussion"
"""
# ***Last line should not contain "|"***
regex_result_others = r"""
    ^\d?[.]?\s?Experiment(s)?$| # "experiment" "experiments"
    ^\d?[.]?\s?Evaluation(s)?((\s)and(\s)comparison(s)?)?$ # "evaluation" "evaluations" "evaluation and comparisons"
"""

# ***Last line should not contain "|"***
regex_extra = r"""
    ^\d?[.]?\s?Discussion(s)?$| # "discussion" "discussions"
    ^\d?[.]?\s?Reference(s)?$| # "reference" "references"
    ^\d?[.]?\s?Conclusion$| # "conclusion"
    ^\d?[.]?\s?Literature(\s)Review$| # "literature review"
    ^\d?[.]?\s?Introduction$| # "introduction"
    ^\d?[.]?\s?Abstract$| # "abstract"
    ^\d?[.]?\s?Limitation(s)?$| # "limitation" "limitations"
    ^\d?[.]?\s?Appendi(x)?(ces)?$| # "appendix" "appendices"
    ^\d?[.]?\s?Bibliography$| # "bibliography"
    ^\d?[.]?\s?Acknowledgement(s)?$| # "acknowledgement" "acknowledgements"
    ^\d?[.]?\s?Tables(\s)and(\s)Figures$ # "tables and figures"
"""

regex_total = (
    regex_method
    + "|"
    + regex_result_only
    + "|"
    + regex_result_others
    + "|"
    + regex_extra
)

In [97]:
# custom_regex = "hi"
custom_regex = None
if custom_regex:
    regex_total += "\n    "
    if custom_regex[0] != "|":
        custom_regex = "|" + custom_regex
    regex_total += custom_regex
# print(regex_total)

# add custom rules
regex_total = re.compile(
    regex_total,
    re.IGNORECASE | re.VERBOSE | re.MULTILINE,
)
regex_method = re.compile(regex_method, re.IGNORECASE | re.VERBOSE)
regex_result_others = re.compile(regex_result_others, re.IGNORECASE | re.VERBOSE)
regex_result_only = re.compile(regex_result_only, re.IGNORECASE | re.VERBOSE)

In [98]:
regex_method.search("4methods")

<re.Match object; span=(0, 8), match='4methods'>

In [99]:
text = get_string_from_text_file()

method_idx = -1
result_idx = -1
result_flag = False  # because "Results" section can be called many names
section_start_point_list = []
for i, match in enumerate(regex_total.finditer(text)):
    section = match.group()
    if regex_method.search(section):
        method_idx = i

    # If there is an exact match with "Results"
    if regex_result_only.search(section):  # if there is an exact match
        result_idx = i
        result_flag = True
    # If not, and if exact match has not been found yet, look for other names
    if regex_result_others.search(section) and not result_flag:
        result_idx = i

    section_start_point_list.append(match.start())
    print(match)
    print([match.group()])
    # break


print(method_idx)
print(result_idx)
print(section_start_point_list)

reading data from document.txt...
string successfully retrieved.

<re.Match object; span=(297, 305), match='Abstract'>
['Abstract']
<re.Match object; span=(1578, 1590), match='Introduction'>
['Introduction']
<re.Match object; span=(10302, 10313), match='Methodology'>
['Methodology']
<re.Match object; span=(22651, 22662), match='Experiments'>
['Experiments']
<re.Match object; span=(25458, 25465), match='Results'>
['Results']
<re.Match object; span=(31333, 31343), match='Conclusion'>
['Conclusion']
<re.Match object; span=(32624, 32634), match='References'>
['References']
2
4
[297, 1578, 10302, 22651, 25458, 31333, 32624]


In [100]:
method_text = text[
    section_start_point_list[method_idx] : section_start_point_list[method_idx + 1]
]
result_text = text[
    section_start_point_list[result_idx] : section_start_point_list[result_idx + 1]
]

In [101]:
result_text

'Results\nExperiment Results on the ImageNet Dataset\nThe main results are illustrated in Figure 3. The images\npositioned on the left with red boxes represent the ground\ntruth images. The second images from the left represent the\nsaliency map reconstructed from EEG signals. The three im-\nages on the right exhibit the three sampling results for the\ngiven pixel-level saliency map with the guidance of sample-\nlevel semantics of EEG signals. Upon comparison with the\nground truth images and the reconstructed saliency maps,\nwe validate that our pixel-level semantics extraction from\nEEG signals successfully captures the color, positional, and\nshape information of viewed images, despite limited seman-\ntic accuracy. Comparing the GT images and three recon-\nstructed samples, it is demonstrated that the latent diffu-\nsion model successfully polishes the decoded saliency mapGT images Sample1 Sample2 Sample3 Saliency Map GT images Sample1 Sample2 Sample3 Saliency MapFigure 3: The main 